# Lab 06 Softmax Zoo_classifier-eager

* Softmax를 사용하여 Zoo 데이터를 활용하여 분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [2]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

(101, 16) (101, 1)


In [3]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([16, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        # tf.nn.softmax computes softmax activations
        # softmax = exp(logits) / reduce_sum(exp(logits), dim)
        logits = tf.matmul(X, self.W) + self.b
        return tf.nn.softmax(logits)
    
    def loss_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=tf.stop_gradient([Y]))
        return cost_i
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(X, Y)
            grads = tape.gradient(loss, self.variables)
            
            return grads
    
    def prediction(self, X, Y):
        pred = tf.argmax(self.softmax_regression(X), 1)
        correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        return accuracy
    
    def fit(self, X, Y, epochs=500, verbose=50):
        optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)
        
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                acc = self.prediction(X, Y).numpy()
                loss = tf.reduce_sum(self.loss_fn(X, Y)).numpy()
                
                print('Loss & Acc at {} epoch {}, {}'.format(i+1, loss, acc))

In [4]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = softmax_classifer(nb_classes)

In [5]:
model.fit(x_data, Y_one_hot)

Loss & Acc at 1 epoch 171.73199462890625, 0.5247524976730347
Loss & Acc at 100 epoch 137.53741455078125, 0.801980197429657
Loss & Acc at 200 epoch 137.0464630126953, 0.801980197429657
Loss & Acc at 300 epoch 136.9971923828125, 0.801980197429657
Loss & Acc at 400 epoch 127.19586181640625, 0.9108911156654358
Loss & Acc at 500 epoch 126.92327880859375, 0.9108911156654358
Loss & Acc at 600 epoch 126.18059539794922, 0.9207921028137207
Loss & Acc at 700 epoch 125.96715545654297, 0.9207921028137207
Loss & Acc at 800 epoch 125.9014663696289, 0.9207921028137207
Loss & Acc at 900 epoch 125.86519622802734, 0.9207921028137207
Loss & Acc at 1000 epoch 125.84115600585938, 0.9207921028137207
